Список нужных классов

In [1]:
class_list = ['person', 'car', 'motorcycle', 'bus', 'truck']

Список классов датасета

In [2]:
coco_classes = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

Список классов не подлежащих обучению

In [3]:
def list_exclude(a, b):
    return [element for element in a if element not in b]

exclude_list = list_exclude(coco_classes, class_list)
exclude_list[:10]

['bicycle',
 'airplane',
 'train',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird']

# Список индексов подлежащих обучению

In [4]:
def list_index_to_exclude(a, b):
    exclude = []
    for each in b:
        print(each)
        for index, element in enumerate(a):
            # print(index, element, sep=': ')
            if each == element:
                exclude.append(str(index))
                break
    return exclude

include_indexes = list_index_to_exclude(coco_classes, class_list)
include_indexes

person
car
motorcycle
bus
truck


['0', '2', '3', '5', '7']

Проверка имспользуемых классов

In [ ]:
print(dataset.default_classes)

Экспорт датасета в формат Yolo

In [ ]:
import fiftyone as fo

export_dir = "/home/mosminin/fiftyone/coco-2017/"
label_field = "ground_truth"  # for example

# The splits to export
splits = ["train", "validation", "test"]

# All splits must use the same classes list
classes = class_list

# Export the splits
for split in splits:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

## Добавление изображений без классов для снижения ложного распознования

# Читаем список файлов для обработки

In [ ]:
from pathlib import Path
data_type = 'train'
files = Path(f'/home/mosminin/fiftyone/coco-2017/labels/{data_type}/').glob('*.txt')
img_path = f'/home/mosminin/fiftyone/coco-2017/images/{data_type}'
name_list = list(map(str, files))
print(len(name_list), name_list[:3], sep='\n')

## Обработчик

# Ищем аннотации с неиспользуемыми классами и переименовываем .txt и соответствующий .jpg

In [ ]:
import multiprocessing, os
from multiprocessing import Value
from time import time, sleep

files = name_list

p_count = multiprocessing.Value('i', 0)
t_count = multiprocessing.Value('f', time())

done_list = []

def process_file(file_name):
    ###
    # t_count.value = time() - t_count.value
    # p_count.value += 1
    # print(str(p_count.value)+'\n')
    ###
    file = open(file_name, 'r')
    lines = file.readlines()
    rename = True
    data = []
    for each in lines:
        # data.append(int(each.split()[0]))
        if each.split()[0] in [exclude_list]:
        # if sum(data) > 0:
            rename = False
    # if rename:        
    #     name = Path(file_name)
    #     # print(f"{name.stem}{name.suffix}_")
    #     name.rename(Path(name.parent, f"{name.stem}{name.suffix}_"))
        
    #     txt_name = Path(file_name)
    #     rname = Path(img_path, f'{txt_name.stem}.jpg')
    #     try:
    #         to_rename = Path(rname.parent, f"{txt_name.stem}{rname.suffix}_")
    #         print(f'name={rname}, txt_name={txt_name.stem}, name_parent={rname.parent} -> {to_rename}\n')
    #         os.rename(rname, to_rename)
    #     except FileNotFoundError:
    #         print(f'{to_rename} not found\n')
    #         pass
    file.close()

with multiprocessing.Pool(4) as pool:
    pool.map(process_file, files, 1);